In [ ]:
import cv2
import numpy as np
import math
import matplotlib.pyplot as plt
from PIL import Image

DATA_PATH = "../../OneDrive - m.titech.ac.jp/Lab/data"

In [ ]:
H_true = np.load(f"{DATA_PATH}/systemMatrix/H_matrix_true.npy")
sample_image = Image.open(f"{DATA_PATH}/sample_image64/Cameraman64.png").convert("L")
sample_image = np.asarray(sample_image).flatten() / 255

Hf = H_true @ sample_image
Hf_img = Hf.reshape(128, 128)
Hf_img = np.clip(Hf_img, 0, 1)
Hf_pil = Image.fromarray((Hf_img * 255).astype(np.uint8), mode="L")

FILENAME = f"GT.png"
fig, ax = plt.subplots(figsize=Hf_img.shape[::-1], dpi=1, tight_layout=True)
ax.imshow(Hf_pil, cmap="gray")
ax.axis("off")
fig.savefig(f"{DATA_PATH}/240724/{FILENAME}", dpi=1)
plt.show()

In [ ]:
IMG_NAME = "Text"
H_SETTING = "hadamard_FISTA_l122_p-5_lmd-100"
# H_SETTING = "gf"
RECONST = "admm_t-1.0"
Original = cv2.imread(f"{DATA_PATH}/sample_image128/{IMG_NAME}.png")
Compared = cv2.imread(f"{DATA_PATH}/240825/reconst/{IMG_NAME}_{H_SETTING}_{RECONST}_edit.png")

# 画素値の読み込み
pixel_value_Ori = Original.flatten().astype(float)
pixel_value_Dis = Compared.flatten().astype(float)

# 画素情報の取得
imageHeight, imageWidth, BPP = Original.shape

# 画素数
N = imageHeight * imageWidth

if BPP == 1:
    # モノクロ画像の場合
    sum = 0
    for i in range(N):
        sum += pow((pixel_value_Ori[i] - pixel_value_Dis[i]), 2)
    MSE = sum / N
    PSNR = 10 * math.log(255 * 255 / MSE, 10)
    print("PSNR", PSNR)
else:
    # RGB画像の場合
    addr = N * BPP

    sumR = 0
    sumG = 0
    sumB = 0

    # 差の2乗の総和を計算
    for i in range(addr):
        if i % 3 == 0:
            sumB += pow((pixel_value_Ori[i] - pixel_value_Dis[i]), 2)
        elif i % 3 == 1:
            sumG += pow((pixel_value_Ori[i] - pixel_value_Dis[i]), 2)
        else:
            sumR += pow((pixel_value_Ori[i] - pixel_value_Dis[i]), 2)

    MSE = (sumR + sumG + sumB) / (3 * N)
    PSNR = 10 * math.log(255 * 255 / MSE, 10)
    print("PSNR", PSNR)


In [ ]:
# SSIM
def SSIM(img1, img2):
    C1 = (0.01 * 255) ** 2
    C2 = (0.03 * 255) ** 2

    img1 = img1.astype(np.float64)
    img2 = img2.astype(np.float64)
    kernel = cv2.getGaussianKernel(11, 1.5)
    window = np.outer(kernel, kernel.transpose())

    mu1 = cv2.filter2D(img1, -1, window)[5:-5, 5:-5]  # valid
    mu2 = cv2.filter2D(img2, -1, window)[5:-5, 5:-5]
    mu1_sq = mu1 ** 2
    mu2_sq = mu2 ** 2
    mu1_mu2 = mu1 * mu2
    sigma1_sq = cv2.filter2D(img1 ** 2, -1, window)[5:-5, 5:-5] - mu1_sq
    sigma2_sq = cv2.filter2D(img2 ** 2, -1, window)[5:-5, 5:-5] - mu2_sq
    sigma12 = cv2.filter2D(img1 * img2, -1, window)[5:-5, 5:-5] - mu1_mu2

    ssim_map = ((2 * mu1_mu2 + C1) * (2 * sigma12 + C2)) / ((mu1_sq + mu2_sq + C1) * (sigma1_sq + sigma2_sq + C2))
    return ssim_map.mean()


ssim = SSIM(Original, Compared)
print("SSIM", ssim)